# Longstaff–Schwartz Monte Carlo – American Options (multi‑asset)

**Zawartość**
1. Symulacja d‑wymiarowego GBM z korelacją.
2. Ogólna procedura LS‑MC dla dowolnych pay‑offów.
3. Payoffs (put, call, call z warunkiem binarnym).
4. Przykłady wyceny.

In [1]:

import numpy as np
import math


## Symulacja ścieżek GBM

In [2]:

def simulate_paths_gbm(S0, r, sigma, corr, T, M, N, seed=None):
    """Symuluje N ścieżek d-wymiarowego GBM (Geometric Brownian Motion)."""
    S0, sigma, corr = map(np.asarray, (S0, sigma, corr))
    d = len(S0)
    if seed is not None:
        np.random.seed(seed)

    dt = T / M
    drift = (r - 0.5 * sigma**2) * dt
    vol = sigma * math.sqrt(dt)

    L = np.linalg.cholesky(corr)

    paths = np.empty((N, M + 1, d))
    paths[:, 0, :] = S0

    for t in range(1, M + 1):
        Z = np.random.normal(size=(N, d)) @ L.T
        paths[:, t, :] = paths[:, t-1, :] * np.exp(drift + vol * Z)

    return paths


## Baza wielomianów (regresja)

In [ ]:

def _build_basis(S, degree=2):
    """Macierz projektująca: 1, S_i, S_i^2, ..."""
    n, d = S.shape
    cols = [np.ones(n)]
    cols += [S[:, j] for j in range(d)]
    if degree >= 2:
        cols += [S[:, j]**2 for j in range(d)]
    if degree >= 3:
        cols += [S[:, j]**3 for j in range(d)]
    return np.column_stack(cols)


## Longstaff–Schwartz

In [ ]:

def lsm_price_multi(S0, r, sigma, corr, T, M, N, payoff_fn, degree=2, seed=0):
    """Wycena American option metodą LS‑MC."""
    paths = simulate_paths_gbm(S0, r, sigma, corr, T, M, N, seed)
    dt = T / M
    disc = math.exp(-r * dt)
    cashflows = payoff_fn(paths[:, -1, :])

    for t in range(M - 1, 0, -1):
        S_t = paths[:, t, :]
        intrinsic = payoff_fn(S_t)
        itm = intrinsic > 0
        if np.any(itm):
            X = _build_basis(S_t[itm], degree)
            Y = cashflows[itm] * disc
            coeff, *_ = np.linalg.lstsq(X, Y, rcond=None)
            continuation = X @ coeff
            exercise = intrinsic[itm] > continuation
            cashflows[np.where(itm)[0][exercise]] = intrinsic[itm][exercise]
        cashflows *= disc
    return cashflows.mean() * disc


## Payoff functions

In [7]:

def payoff_put(K, idx=0):
    return lambda S: np.maximum(K - S[:, idx], 0.0)

def payoff_call(K, idx=0):
    return lambda S: np.maximum(S[:, idx] - K, 0.0)

def payoff_call_binary(K, idx_pay=0, idx_cond=1, H=1.0):
    return lambda S: np.maximum(S[:, idx_pay] - K, 0.0) * (S[:, idx_cond] > H)


## Przykłady wyceny

In [6]:

M, N = 50, 50_000

put_price = lsm_price_multi(
    S0=[36], r=0.06, sigma=[0.20], corr=[[1]], T=1.0,
    M=M, N=N, payoff_fn=payoff_put(K=40), seed=42)
print(f"American put ≈ {put_price:.4f}")

call_price = lsm_price_multi(
    S0=[36], r=0.06, sigma=[0.20], corr=[[1]], T=1.0,
    M=M, N=N, payoff_fn=payoff_call(K=36), seed=42)
print(f"American call ≈ {call_price:.4f}")

basket_price = lsm_price_multi(
    S0=[50, 30],
    r=0.05,
    sigma=[0.25, 0.20],
    corr=[[1, 0.3], [0.3, 1]],
    T=1.0,
    M=M, N=N,
    payoff_fn=payoff_call_binary(K=55, idx_pay=0, idx_cond=1, H=32),
    seed=42)
print(f"Binary‑conditioned call ≈ {basket_price:.4f}")


American put ≈ 4.4657
American call ≈ 3.9332
Binary‑conditioned call ≈ 2.7952
